# 03 — Segmentation

## Goals
- Segment customers using the feature table and the segmentation configuration.
- Map segments to personas and perks to produce the rewards outputs.

## Inputs
- `config/segmentation.yaml` (segmentation settings; we copy it to a deterministic output folder).
- `config/perks.yaml` (segment-to-perk mapping).
- `artifacts/feature_engineering/customer_features.parquet` from feature engineering.

## Outputs
- `artifacts/segmentation/segment_assignments.parquet`
- `artifacts/segmentation/segment_summary.parquet`
- `artifacts/segmentation/decision_report.md`
- `artifacts/segmentation/customer_perks.csv`


In [ ]:
from pathlib import Path

import yaml

from traveltide.perks.mapping import write_customer_perks
from traveltide.segmentation.run import run_segmentation_job

segmentation_outdir = Path("artifacts/segmentation")
segmentation_outdir.mkdir(parents=True, exist_ok=True)

segmentation_cfg = yaml.safe_load(Path("config/segmentation.yaml").read_text())
segmentation_cfg["output"]["outdir"] = str(segmentation_outdir)

segmentation_cfg_path = segmentation_outdir / "segmentation.yaml"
segmentation_cfg_path.write_text(yaml.safe_dump(segmentation_cfg, sort_keys=False))

run_dir = run_segmentation_job(config_path=str(segmentation_cfg_path))

perks_path = write_customer_perks(
    assignments_path=str(run_dir / "segment_assignments.parquet"),
    config_path="config/perks.yaml",
    out_path=str(segmentation_outdir / "customer_perks.csv"),
)

run_dir, perks_path
